# K210分类模型的训练

In [ ]:
# https://aistudio.baidu.com/aistudio/projectdetail/2410414?contributionType=1

##  获取git代码构建环境

In [1]:
# %%bash
# if [ -d "/home/aistudio/work/k210" ]; then
#   rm -f -r "/home/aistudio/work/k210"
# fi
# # git代码没更新，就没必要删除重新获取

In [2]:
# %%bash
# cd /home/aistudio/work/
# git clone https://gitee.com/ai-liam/k210
# #git clone https://github.com/ai-liam/k210
## 下载过就不必要下载

In [3]:
# %%bash
# ls /home/aistudio/work/k210/k210_train

In [4]:
!mkdir /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists


In [5]:
## 安装过就必要在安装
#pip install -r /home/aistudio/work/k210/k210_train/requirements.txt -i https://mirrors.aliyun.com/pypi/simple/ -t /home/aistudio/external-libraries

## 上传和设置数据源目录

In [6]:
# 把你的 xx.zip 数据上到 work目录，【注意，解压得到为 images】参考例子数据
upload_zip = f"/home/aistudio/work/k210/datasets/classifier_datasets.zip"# demo数据

## 开始训练

In [7]:
import sys 
sys.path.append('/home/aistudio/external-libraries')
sys.path.append('/home/aistudio/work/k210/k210_train')

!ls

2410414.ipynb  data  external-libraries  work


In [8]:
import os, sys
import time


In [ ]:
from train.classifier_v1 import classifier_train
from train.classifier.instance import config

config.classifier_train_epochs = 40 # 修改训练轮数 40

In [ ]:
cur_time = time.strftime("%Y-%m-%d_%H_%M", time.localtime())
print(cur_time)

2021-09-26_16_40


In [ ]:
root_dir = "/home/aistudio/work/k210"
ubuntu_path = f"out/imgn_{cur_time}" #用来mac中生成kmodel
temp_saves_dir = f"{root_dir}/ubuntu/{ubuntu_path}/" #保存文件目录
datasets_dir =  upload_zip #数据源

print("root_dir:",root_dir)
print("temp_saves_dir:",temp_saves_dir)
print("datasets_dir:",datasets_dir)

root_dir: /home/aistudio/work/k210
temp_saves_dir: /home/aistudio/work/k210/ubuntu/out/imgn_2021-09-26_16_40/
datasets_dir: /home/aistudio/work/k210/datasets/classifier_datasets.zip


In [ ]:
# 配置文件：train.classifier.instance
#result,_,model_path ,labels = classifier_train(datasets_dir=datasets_dir,temp_datasets_dir=temp_saves_dir,config=config)

result,_,model_path ,labels = classifier_train(datasets_zip_path=datasets_dir,temp_datasets_dir=temp_saves_dir,config=config)

select gpu fail:NVML Shared Library Not Found


2021-09-26 16:40:05,575 - [INFO]:  no GPU, will use [CPU]
2021-09-26 16:40:07,385 - [INFO]:  train, labels:['1', '0']
2021-09-26 16:40:07,386 - [DEBUG]:  train, datasets dir:/home/aistudio/work/k210/ubuntu/out/imgn_2021-09-26_16_40//images


self.datasets_dir: /home/aistudio/work/k210/ubuntu/out/imgn_2021-09-26_16_40//images


/home/aistudio/external-libraries/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2021-09-26 16:40:08,346 - [INFO]:  train data:480, valid data:120


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 24)      648       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 24)      96        
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 24)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 24)      216       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 24)      96    

/home/aistudio/external-libraries/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-09-26 16:40:08,785 - [INFO]:  train start
2021-09-26 16:40:08,786 - [INFO]:  progress: 1.0%, train start
2021-09-26 16:40:08,838 - [INFO]:  epoch 0 start


Epoch 1/40
 6/96 [>.............................] - ETA: 59s - loss: 0.7232 - accuracy: 0.566

In [ ]:
print("model_path:",model_path)
print("labels:",labels)

## 生成k210专用模型 kmodel

In [2]:
'''
上传 ncc文件
下载：https://github.com/kendryte/nncase/releases/tag/v0.1.0-rc5

unzip -o -d /home/aistudio/work/ /home/aistudio/work/ncc-linux-x86_64.zip
目录为：
~/work$ ls
ncc-linux-x86_64  ncc-linux-x86_64.zip
'''

'\n上传 ncc文件\n下载：https://github.com/kendryte/nncase/releases/tag/v0.1.0-rc5\n\nunzip -o -d /home/aistudio/work/ /home/aistudio/work/ncc-linux-x86_64.zip\n目录为：\n~/work$ ls\nncc-linux-x86_64  ncc-linux-x86_64.zip\n'

In [ ]:
import os

ncc_path = "/home/aistudio/work/ncc-linux-x86_64/ncc" # 百度平台
cmd_linux = f"{ncc_path} -i tflite -o k210model --dataset {temp_saves_dir}sample_images {temp_saves_dir}m.tflite {temp_saves_dir}result/m.kmodel"
print(cmd_linux)

# -i tflite -o k210model --dataset ./$1/sample_images ./$1/m.tflite ./$1/result/m.kmodel

In [ ]:
f=os.popen(cmd_linux)
print(f.read())

In [ ]:
# 生成的文件
cmd = f"ls {temp_saves_dir}result"
print("cmd:",cmd)
f=os.popen(cmd)
print(f.read())

## 生成k210程序boot.py

In [ ]:
target_file = f"{temp_saves_dir}result/boot.py"
copy_file = f"cp -f {root_dir}/k210_train/data/template/imgn_boot.py {target_file}"
print(copy_file)

In [ ]:
f=os.popen(copy_file)
print(f.read())

In [ ]:
# 更新 labels 
result_boot_py_path = target_file

img_size = [224,224]
#labels = ["right", "left", "forward"]
replace = 'labels = ["{}"]'.format('", "'.join(labels))

with open(result_boot_py_path) as f:
            boot_py = f.read()
with open(result_boot_py_path, "w") as f:
            target = 'labels = ["0", "1"] # labels'
            boot_py = boot_py.replace(target, replace)
            target = 'img_size = [224,224] # img_size'
            replace = 'img_size = [{},{}]'.format(img_size[0], img_size[1])
            boot_py = boot_py.replace(target, replace)
            f.write(boot_py)

## 更新到k210 的cube硬件上运行

In [ ]:
# 生成的文件
cmd = f"ls {temp_saves_dir}result"
print("cmd:",cmd)
f=os.popen(cmd)
print(f.read())

In [ ]:
'''
把 
├── result
│   ├── boot.py
│   └── m.kmodel

的 boot.py ，m.kmodel 复制到 sd 卡的根目录下（或者 models/m8/目录下）
重新启动硬件，就可以运行了
boot.py 代码中核实模型的路径是否正确
'''